### Этап 2. Чат-бот.

In [1]:
import re
import os
import requests
import annoy
import pickle
import numpy as np
import time

from gensim.models import Word2Vec
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import logging

import telebot
from telebot import types

from geopy import geocoders
from yaweather import Russia, YaWeather

import wikipedia

In [2]:
bot = telebot.TeleBot("<YOUR BOT TOKEN>")

In [3]:
# погода

#функция получения погоды
def get_weather(message):
    city = message.text
    try:
        geolocator = geocoders.Nominatim(user_agent="telebot")
        latitude = geolocator.geocode(city).latitude
        if latitude == "ZERO_RESULTS":
            forecast = f'К сожалению я не нашел в прогнозе город {city}'
            bot.send_message(message.chat.id, forecast)
            #bot.register_next_step_handler(message, startCommand) 
        else:    
            longitude = geolocator.geocode(city).longitude
            y = YaWeather(api_key='<YOUR API KEY>')
            res = y.forecast((latitude, longitude))
            forecast = f'В городе {city} сейчас {res.fact.temp} °C, oщущается как {res.fact.feels_like} °C. Погодные явления: {res.fact.condition}. Влажность {res.fact.humidity}'
            bot.send_message(message.chat.id, forecast) 
            time.sleep(2) 
            bot.register_next_step_handler(message, startCommand) 
    except Exception as e:
        print(repr(e))

In [4]:
# википедия

wikipedia.set_lang("ru")

def get_wiki(message):
    try:
        message_text = message.text
        get_page = wikipedia.page(message_text)
        wikitext = get_page.content[:1000]
        wikimas = wikitext.split('.')
        wikimas = wikimas[:-1]
        wikitext2 = ''
        for x in wikimas:
            if not('==' in x):
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        bot.send_message(message.chat.id,f'{wikitext2}\n') 
        time.sleep(2) 
        bot.register_next_step_handler(message, startCommand) 
    except Exception as e:
        print(repr(e))

In [5]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
answer_model = Word2Vec.load("w2v_model")
answer_index = annoy.AnnoyIndex(100 ,'angular')
answer_index.load('speaker.ann')
index_map = pickle.load(open("index_map",'rb'))

def find_answer(message):
    question = message.text
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in answer_model.wv:
            vector += answer_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    query_index = answer_index.get_nns_by_vector(vector, 1)
    bot.send_message(message.chat.id, index_map[query_index[0]])
    time.sleep(2) 
    bot.register_next_step_handler(message, startCommand) 
    

In [7]:
#курс валют

def get_exchange(message):
    try:
        if message.text == 'USD':
            data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
            bot.send_message(message.chat.id, data['Valute']['USD']['Value'])
            time.sleep(2) 
            bot.register_next_step_handler(message, startCommand) 
        elif message.text == 'EUR':
            data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
            bot.send_message(message.chat.id, data['Valute']['EUR']['Value'])
            time.sleep(2) 
            bot.register_next_step_handler(message, startCommand) 
        else:
            bot.send_message(message.chat.id, 'Эту валюту не могу показать. Знаю только USD и EUR')
            time.sleep(2) 
            bot.register_next_step_handler(message, startCommand)
    
    except Exception as e:
        print(repr(e))

In [8]:
@bot.message_handler(commands=['start'])
def startCommand(message):

    markup=types.InlineKeyboardMarkup(row_width=1)
    buttonChat = types.InlineKeyboardButton('Задай мне вопрос', callback_data='Chat')
    buttonExchange = types.InlineKeyboardButton('Курс валют', callback_data='Exchange')
    buttonWeather = types.InlineKeyboardButton('Погода', callback_data='Weather')
    buttonWiki = types.InlineKeyboardButton('Википедия', callback_data='Wiki')
    markup.add(buttonChat, buttonExchange, buttonWeather, buttonWiki)
    bot.send_message(message.chat.id,'Выбери возможное действие ниже :)', reply_markup=markup)
    
               
@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
   try:
      if call.message:
         if call.data == 'Chat':
            msg = bot.send_message(call.message.chat.id, 'Я молодой бот и только учусь). Задавай вопрос!\nДля вызова меню после моего ответа напиши любое слово')
            bot.register_next_step_handler(msg, find_answer)
         elif call.data == 'Exchange':
            #bot.send_message(call.message.chat.id, 'Тут будет курс валют\nДля вызова меню после моего ответа напиши любое слово')
            msg = bot.send_message(call.message.chat.id, 'Покажу курс доллара или евро в рублях. Введите USD или EUR.\nДля вызова меню после моего ответа напиши любое слово')
            bot.register_next_step_handler(msg, get_exchange)
         elif call.data == 'Weather':
            msg = bot.send_message(call.message.chat.id, 'Введите название города\nДля вызова меню после моего ответа напиши любое слово')
            bot.register_next_step_handler(msg, get_weather)
         elif call.data == 'Wiki':
            msg = bot.send_message(call.message.chat.id, 'Что ищем на Wikipedia?\nДля вызова меню после моего ответа напиши любое слово')
            bot.register_next_step_handler(msg, get_wiki)
			
   except Exception as e:
      print(repr(e))


bot.polling(none_stop=True, interval=0)
#bot.infinity_polling()